Subjest
===
Use LSTM scheme to predict the future. Create a new directory, named "model", in which the trained model should be stored here.




Requirements
---
1. warktermark, An IPython magic extension for printing date and time stamps, version numbers, and hardware information. (install by pip)
- tensorflow
- keras


In [ ]:
%load_ext watermark
%watermark -v -p tensorflow,sklearn,keras,scipy,pandas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
dir_name='../COVID-19-master/csse_covid_19_data/csse_covid_19_time_series/'
url = dir_name+"/time_series_covid19_confirmed_global.csv"
df_confirmed = pd.read_csv(url)

In [ ]:
df_confirmed.head()

In [ ]:
def df_washing(df_,col="confirmed",country="Taiwan*"):
    """
    simple washing for df
    """
    df=df_.copy()
    
    # retrieve the data
    df_1 = df[df["Country/Region"] == country]

    ## structuring times eries data
    df_2 = pd.DataFrame(df_1[df_1.columns[4:]].sum(),columns=[col])
    df_2.index = pd.to_datetime(df_2.index)#,format='%m/%d/%y')
    return df_2

In [ ]:
country="Taiwan*"

df_confirmed2 = df_washing(df_confirmed)
df_confirmed2.tail()

In [ ]:
## for cases of dead

df_dead = pd.read_csv(dir_name+"time_series_covid19_deaths_global.csv")

In [ ]:
df_dead.head()

In [ ]:
df_dead2 = df_washing(df_dead,col="dead")


In [ ]:
### for cases of recovered
df_recovered = pd.read_csv(dir_name+"time_series_covid19_recovered_global.csv")

df_recovered2 = df_washing(df_recovered,col="recovered")


In [ ]:
## join confirmed with dead
df_conf_dead = df_confirmed2.join(df_dead2,how = "inner")


In [ ]:
## join confirmed+dead with recovered
df_all = df_conf_dead.join(df_recovered2,how = "inner")
df_all.tail()

In [ ]:
df_all.plot(figsize=(14,6),kind='line', marker='o',\
            title="COVID-19 statistics at %s" %country)
plt.grid(True)

In [ ]:
# up to 2020-04-06
df_new = df_confirmed2[["confirmed"]]
df_new.tail()

In [ ]:
# daily data and i want to predict 5 days afterwards
len(df_new)

In [ ]:
x = len(df_new)-5
x

In [ ]:
train=df_new.iloc[:x]
test = df_new.iloc[x:]
train.tail()

Data ranges too wide to keep stationary; one of the methods is to convert them among `[0,1]`. 

In [ ]:
##scale or normalize data as the data is too skewed
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
scaler.fit(train) #find max value 

In [ ]:
scaled_train = scaler.transform(train)#and divide every point by max value
scaled_test = scaler.transform(test)
print(scaled_train[-5:])

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(scaled_train)
plt.grid(True)

In [ ]:
## feed in batches [t1,t2,t3,t4,t5] --> y=t6

from keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
scaled_train.shape

In [ ]:
## how to decide num of inputs , 
n_input = 5  ## number of steps
n_features = 1 ## number of features you want to predict (for univariate time series n_features=1)
generator = TimeseriesGenerator(scaled_train,scaled_train,length = n_input,batch_size=1)

In [ ]:
#62
len(scaled_train)

In [ ]:
#57
len(generator)

In [ ]:
scaled_train[:6]

In [ ]:
generator[0]

In [ ]:
## above takes 5 inputs and predicts next point in scaled_train
## smaller batch size leads to better trainig for time series

Deep Learning Moel
---

```
Layer1 (Input Layer)   ➡︎  Layer2   ➠   ...    ➡︎ Layer (n-1)  ➡︎  Layer n (output Layer)
      X1              P1    X2                     X(n-1)    Pn   y = Xn

    Pi = Processing: AXi+bi ➧ Activate-i (relu, gelu, etc)
```

In [ ]:
import tensorflow as tf
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects
from tensorflow.keras.callbacks import EarlyStopping

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation

In [ ]:
def gelu(x):
    """Gaussian Error Linear Unit.
    This is a smoother version of the RELU.
    Original paper: https://arxiv.org/abs/1606.08415
    Args:
        x: float Tensor to perform activation.
    Returns:
        `x` with the GELU activation applied.
    """
    cdf = 0.5 * (1.0 + tf.tanh(
        (np.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3)))))
    return x * cdf

In [ ]:
X= np.linspace(-3,3,101)
Y=gelu(X)
Y_relu=np.where(X<0,0,X)

plt.figure(figsize=(6,6))
plt.title("Activate function relu and gelu")
plt.plot(X,Y,label="gelu")
plt.plot(X,Y_relu,label="relu")
plt.grid(True)
plt.xlim([-3,3])
plt.ylim([-2,4])
plt.legend()

In [ ]:
np.where?

In [ ]:
x0=np.linspace(-2.5,2,101)
y0=x0*x0*x0/2-2*x0*x0-3*x0+4.5#3*np.sin(3*x0-1)
y_relu=np.where(y0<0,0,y0)
y_gelu= gelu(y0)
plt.figure(figsize=[6,8])
plt.title("How Activation works in Deep earning Model")
plt.plot(x0,y0,label="$x^3/2-2x^2-3x+4.5$",color="blue")
plt.plot(x0,y_relu,label='relu')
plt.plot(x0,y_gelu,label='gelu',color="red")
plt.xlim([-3,3])
plt.ylim([-6,6])

plt.legend()

In [ ]:
import tensorflow as tf
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects

def custom_gelu(x):    
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))

get_custom_objects().update({'custom_gelu': Activation(custom_gelu)})

```
LSTM:  Long Short Term Memory Networks (LSTM) models           

           150 neurons      75 neurons      1 neuron
  input ➨    LSTM       ➨   Dense Layer  ➨  output  
```  
1. Input and output: 5 daily data are used to predict the afterward one day data:
$$\text{Input: }\mathbf{X}=[t_i,t_{i+1},t_{i+2},t_{i+3},t_{i+4}]\Rightarrow \text{Output: }\mathbf{y}=[t_{i+5}]$$
2. As above, processing in each layler is by 
  a) firstly, Linear regression;
  b) and by activation as follows:
  
$$\mathbf{X_{n\times5}\Rightarrow X_{n\times5}w_{5\times1}+b_{n\times1}{{\text{Activation}\atop\Rightarrow }}y_{n\times1}}  $$

by optimizer and loss (measurement of loss).
3. Deep learning introduces additional neurons layer within the sequential model as follows:
```
         hidden layer
             ○
         ⤢  
 
      ●      ...     ➠ Output
         ⤡
             ○
```
3. LSTM also keeps the long-term memory infos except the short-term memory.
4. Activatation is an efficient nonliner mechinization, relu or gelu etc;

In [ ]:
def LSTM_model(df_):
    
    print("1. Splite Data  into train/test sets ...\n")
    df=df_.copy()
    x = len(df)-5
    train_1=df.iloc[:x]
    test_1 = df.iloc[x:]
    print("2. Scale data into [0,1] ...\n")
    scaler = MinMaxScaler()
    scaler.fit(train_1)
    scaled_train_1 = scaler.transform(train_1)
    scaled_test_1 = scaler.transform(test_1)
    
    print("3. Combine into TimeSeries Gererator for keras ...\n")
    n_input = 5  ## number of steps
    n_features = 1 ## number of features you want to predict (for univariate time series n_features=1)
    generator = TimeseriesGenerator(scaled_train_1,scaled_train_1,length = n_input,batch_size=1)
    
    print("4. Setup LSTM Model ... \n")
    model = Sequential()
    model.add(LSTM(150,activation=custom_gelu,input_shape=(n_input,n_features)))
    model.add(Dropout(0.3))
    #model.add(Dense(75, activation="gelu"))
    model.add(Dense(75, activation=custom_gelu))
    model.add(Dense(units=1))
    model.compile(optimizer="adam",loss="mse")
    
    print("5. add validate set ...\n")
    
    val_ind=len(train_1)-5
    validation_set_1 = np.append(scaled_train_1[val_ind],scaled_test_1)
    validation_set_1=validation_set_1.reshape(6,1)
    validation_gen = TimeseriesGenerator(validation_set_1,validation_set_1,length=5,batch_size=1)
    
    print("6, add early stop ...\n")
    early_stop = EarlyStopping(monitor='val_loss',patience=20,restore_best_weights=True)
    print("7. training model ... \n")
    model.fit_generator(generator,validation_data=validation_gen,epochs=100,\
                        callbacks=[early_stop],steps_per_epoch=10,verbose=0)
    #pd.DataFrame(model.history.history).plot(title="loss vs epochs curve",figsize=(10,6))
    
    
    print("8. Predict the next 7 date data ...\n") 
    ## holding predictions
    test_prediction = []

    ##last n points from training set
    first_eval_batch = scaled_train_1[-n_input:]
    current_batch = first_eval_batch.reshape(1,n_input,n_features)
    
    for i in range(len(test_1)+7):
        current_pred = model.predict(current_batch)[0]
        test_prediction.append(current_pred)
        current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)
    
    true_prediction = scaler.inverse_transform(test_prediction)
    
    time_series_array = test_1.index
    for k in range(0,7):
        time_series_array = time_series_array.append(time_series_array[-1:] + pd.DateOffset(1))
    df_forecast = pd.DataFrame(columns=["confirmed","confirmed_predicted"],index=time_series_array)
    df_forecast.loc[:,"confirmed_predicted"] = true_prediction[:,0]
    df_forecast.loc[:,"confirmed"] = test_1["confirmed"]
    
    return df_forecast

In [ ]:
df_confirmed_pred=LSTM_model(df_new)

In [ ]:
df_confirmed_pred

Summary
---
...